In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multitest import multipletests

os.chdir("..")
root_path = os.getcwd()


In [2]:
path_csv = os.chdir(root_path+'/Results_StandardBagging')
df_standard = pd.read_csv('Results_StandardBagging_20_30_50_100_150_200_ensembles.csv')

path_csv = os.chdir(root_path+'/MixedBagging/Adapted_results')
df_mixed = pd.read_csv('Results_MixedBagging_20_30_50_100_150_200_ensembles.csv') # from 20 n_trees


path_csv = os.chdir(root_path+'/Results_general_algorithm')
df_cdb = pd.read_csv('Results_CDB_Filter6Parameters_20_30_50_100_150_200_ensembles.csv')



In [3]:

# Me quedo con el número de ensamblados que quiero y selecciono los mejores parámetros

def Results_n_ensembles(df_cdb, df_mixed, df_standard, n_trees):

    # Filter
    df_cdb_trees = df_cdb[df_cdb['n_ensemble'] == n_trees]
    df_standard_trees = df_standard[df_standard['n_ensemble'] == n_trees]
    df_mixed_trees = df_mixed[df_mixed['n_trees'] == (n_trees+1)]

    table_comparison = pd.DataFrame(columns=['Dataset', 'Standard_Bag_mean', 'Standard_Bag_std'])
    table_comparison['Dataset'] = df_standard_trees['Dataset']
    table_comparison['Standard_Bag_mean'] = df_standard_trees["accuracy_mean"]
    table_comparison['Standard_Bag_std'] = df_standard_trees["accuracy_std"]
    table_comparison.set_index('Dataset', inplace=True)

    ## Grouped and incremented
    df_mixed_aux = df_mixed_trees.loc[df_mixed_trees['model'] == 'Grouped_Mixed_Bagging',
    ['Dataset', 'accuracy_mean', 'accuracy_std']]
    df_mixed_aux.columns = ['Dataset', 'Grouped_accuracy_mean', 'Grouped_accuracy_std']
    df_mixed_inc = df_mixed_trees.loc[df_mixed_trees['model'] == 'Incremental_Mixed_Bagging',
    ['Dataset', 'accuracy_mean', 'accuracy_std']]
    df_mixed_inc.columns = ['Dataset', 'Incremental_accuracy_mean', 'Incremental_accuracy_std']
    df_mixed_aux.set_index('Dataset', inplace=True)
    df_mixed_inc.set_index('Dataset', inplace=True)

    table_comparison = table_comparison.join(df_mixed_aux[['Grouped_accuracy_mean', 'Grouped_accuracy_std']])
    table_comparison = table_comparison.join(
        df_mixed_inc[['Incremental_accuracy_mean', 'Incremental_accuracy_std']])
    table_comparison.columns = ['Standard_Bag_mean', 'Standard_Bag_std', 'Grouped_Bag_mean', 'Grouped_Bag_std',
                                'Incre_Bag_mean', 'Incre_Bag_std']

    ## Best parameters per dataset in CDB
    best_param = df_cdb_trees.loc[df_cdb_trees.groupby(["Dataset", "weights"])["accuracy_mean"].idxmax()]

    # Hostility
    df_aux = best_param.loc[
        best_param['weights'] == 'Hostility', ['Dataset', 'accuracy_mean', 'accuracy_std']]
    df_aux.set_index('Dataset', inplace=True)
    table_comparison = table_comparison.join(df_aux[['accuracy_mean', 'accuracy_std']])
    table_comparison.rename(columns={'accuracy_mean': 'CDB_Host_mean',
                                     'accuracy_std': 'CDB_Host_std'}, inplace=True)
    # kDN
    df_aux = best_param.loc[best_param['weights'] == 'kDN', ['Dataset', 'accuracy_mean', 'accuracy_std']]
    df_aux.set_index('Dataset', inplace=True)
    table_comparison = table_comparison.join(df_aux[['accuracy_mean', 'accuracy_std']])
    table_comparison.rename(columns={'accuracy_mean': 'CDB_kDN_mean',
                                     'accuracy_std': 'CDB_kDN_std'}, inplace=True)
    # CLD
    df_aux = best_param.loc[best_param['weights'] == 'CLD', ['Dataset', 'accuracy_mean', 'accuracy_std']]
    df_aux.set_index('Dataset', inplace=True)
    table_comparison = table_comparison.join(df_aux[['accuracy_mean', 'accuracy_std']])
    table_comparison.rename(columns={'accuracy_mean': 'CDB_CLD_mean',
                                     'accuracy_std': 'CDB_CLD_std'}, inplace=True)
    # LSC
    df_aux = best_param.loc[best_param['weights'] == 'LSC', ['Dataset', 'accuracy_mean', 'accuracy_std']]
    df_aux.set_index('Dataset', inplace=True)
    table_comparison = table_comparison.join(df_aux[['accuracy_mean', 'accuracy_std']])
    table_comparison.rename(columns={'accuracy_mean': 'CDB_LSC_mean',
                                     'accuracy_std': 'CDB_LSC_std'}, inplace=True)
    # N1
    df_aux = best_param.loc[best_param['weights'] == 'N1', ['Dataset', 'accuracy_mean', 'accuracy_std']]
    df_aux.set_index('Dataset', inplace=True)
    table_comparison = table_comparison.join(df_aux[['accuracy_mean', 'accuracy_std']])
    table_comparison.rename(columns={'accuracy_mean': 'CDB_N1_mean',
                                     'accuracy_std': 'CDB_N1_std'}, inplace=True)
    # N2
    df_aux = best_param.loc[best_param['weights'] == 'N2', ['Dataset', 'accuracy_mean', 'accuracy_std']]
    df_aux.set_index('Dataset', inplace=True)
    table_comparison = table_comparison.join(df_aux[['accuracy_mean', 'accuracy_std']])
    table_comparison.rename(columns={'accuracy_mean': 'CDB_N2_mean',
                                     'accuracy_std': 'CDB_N2_std'}, inplace=True)
    # DCP
    df_aux = best_param.loc[best_param['weights'] == 'DCP', ['Dataset', 'accuracy_mean', 'accuracy_std']]
    df_aux.set_index('Dataset', inplace=True)
    table_comparison = table_comparison.join(df_aux[['accuracy_mean', 'accuracy_std']])
    table_comparison.rename(columns={'accuracy_mean': 'CDB_DCP_mean',
                                     'accuracy_std': 'CDB_DCP_std'}, inplace=True)
    # TD_U
    df_aux = best_param.loc[best_param['weights'] == 'TD_U', ['Dataset', 'accuracy_mean', 'accuracy_std']]
    df_aux.set_index('Dataset', inplace=True)
    table_comparison = table_comparison.join(df_aux[['accuracy_mean', 'accuracy_std']])
    table_comparison.rename(columns={'accuracy_mean': 'CDB_TD_U_mean',
                                     'accuracy_std': 'CDB_TD_U_std'}, inplace=True)
    # F1
    df_aux = best_param.loc[best_param['weights'] == 'F1', ['Dataset', 'accuracy_mean', 'accuracy_std']]
    df_aux.set_index('Dataset', inplace=True)
    table_comparison = table_comparison.join(df_aux[['accuracy_mean', 'accuracy_std']])
    table_comparison.rename(columns={'accuracy_mean': 'CDB_F1_mean',
                                     'accuracy_std': 'CDB_F1_std'}, inplace=True)


    table_comparison_mean = table_comparison.loc[:,
                            table_comparison.columns[table_comparison.columns.str.endswith('mean')]]

    return table_comparison, table_comparison_mean



In [4]:
methods = ['Standard_Bag_mean','Grouped_Bag_mean','Incre_Bag_mean','CDB_Host_mean',
           'CDB_kDN_mean','CDB_CLD_mean','CDB_LSC_mean','CDB_N1_mean','CDB_N2_mean',
           'CDB_DCP_mean','CDB_TD_U_mean','CDB_F1_mean']

def rank_df(df_mean, method, methods=methods):
    rank_index = [i for i in methods if i != method]
    wtl_df = pd.DataFrame(index=rank_index,
                          columns=['Wins', 'Ties', 'Losses'])
    for i in rank_index:
        wtl_df.loc[i, 'Wins'] = sum(df_mean[method] > df_mean[i])
        wtl_df.loc[i, 'Losses'] = sum(df_mean[method] < df_mean[i])
        wtl_df.loc[i, 'Ties'] = len(df_mean) - wtl_df.loc[i, 'Wins'] - wtl_df.loc[i, 'Losses']
    wtl_plot = wtl_df.copy()
    wtl_plot['Ties'] = wtl_df['Wins'] + wtl_df['Ties']
    wtl_plot['Losses'] = wtl_plot['Ties'] + wtl_df['Losses']

    return wtl_df, wtl_plot

In [5]:

### Sacamos los resultados para distintos cortes
n_trees_v = [19,29,49,99,149,199]

total_dict_str = {}
for n_trees in n_trees_v:
    n_trees_str = str(n_trees+1)
    _, table_comparison_n_trees = Results_n_ensembles(df_cdb, df_mixed, df_standard, n_trees)
    total_dict_ntrees = {}
    total_dict_ntrees_str = {}
    for method_i in methods:
        wtl_df, _ = rank_df(table_comparison_n_trees, method_i, methods=methods)
        wtl_df_str = pd.DataFrame()
        wtl_df_str['wtl'] = "(" + wtl_df.apply(lambda row: ','.join(row.values.astype(str)), axis=1) + ")"
        total_dict_ntrees[method_i] = wtl_df
        total_dict_ntrees_str[method_i] = wtl_df_str
    total_dict_str[n_trees_str] = total_dict_ntrees_str



# n_trees_index = list(total_dict_str.keys())
methods_CM = methods[3:]
dict_to_table = {}
for method_i in methods_CM:
    df_to_append = pd.DataFrame()
    for k in total_dict_str.keys():
        df_to_append = pd.concat([df_to_append,pd.DataFrame(total_dict_str[k][method_i].T)])
    df_to_append.index = list(total_dict_str.keys())
    dict_to_table[method_i] = df_to_append



In [6]:
dict_to_table['CDB_Host_mean']

,Standard_Bag_mean,Grouped_Bag_mean,Incre_Bag_mean,CDB_kDN_mean,CDB_CLD_mean,CDB_LSC_mean,CDB_N1_mean,CDB_N2_mean,CDB_DCP_mean,CDB_TD_U_mean,CDB_F1_mean
20,"(41,3,0)","(39,2,3)","(37,2,5)","(22,6,16)","(19,4,21)","(25,4,15)","(28,3,13)","(17,3,24)","(20,4,20)","(24,4,16)","(21,3,20)"
30,"(40,3,1)","(38,2,4)","(37,2,5)","(27,5,12)","(15,5,24)","(21,5,18)","(25,3,16)","(24,3,17)","(21,5,18)","(22,6,16)","(23,4,17)"
50,"(41,3,0)","(37,2,5)","(36,4,4)","(24,9,11)","(17,5,22)","(24,4,16)","(26,5,13)","(24,5,15)","(21,6,17)","(21,5,18)","(24,5,15)"
100,"(38,4,2)","(35,2,7)","(32,3,9)","(21,7,16)","(16,8,20)","(23,3,18)","(25,5,14)","(22,6,16)","(17,4,23)","(20,3,21)","(23,4,17)"
150,"(39,4,1)","(36,2,6)","(32,3,9)","(21,7,16)","(13,8,23)","(21,4,19)","(28,4,12)","(18,7,19)","(16,4,24)","(21,6,17)","(20,7,17)"
200,"(40,3,1)","(34,2,8)","(31,2,11)","(23,5,16)","(14,8,22)","(22,5,17)","(28,6,10)","(25,4,15)","(24,4,16)","(21,6,17)","(22,7,15)"


In [7]:
dict_to_table['CDB_kDN_mean']

,Standard_Bag_mean,Grouped_Bag_mean,Incre_Bag_mean,CDB_Host_mean,CDB_CLD_mean,CDB_LSC_mean,CDB_N1_mean,CDB_N2_mean,CDB_DCP_mean,CDB_TD_U_mean,CDB_F1_mean
20,"(41,3,0)","(39,2,3)","(36,2,6)","(16,6,22)","(14,4,26)","(17,4,23)","(19,4,21)","(16,6,22)","(21,3,20)","(18,6,20)","(17,5,22)"
30,"(38,3,3)","(37,2,5)","(36,3,5)","(12,5,27)","(8,7,29)","(13,5,26)","(21,4,19)","(19,5,20)","(11,6,27)","(17,4,23)","(19,4,21)"
50,"(40,3,1)","(37,2,5)","(34,3,7)","(11,9,24)","(17,3,24)","(18,4,22)","(22,6,16)","(21,4,19)","(19,3,22)","(21,3,20)","(19,5,20)"
100,"(41,3,0)","(32,3,9)","(30,2,12)","(16,7,21)","(11,4,29)","(20,3,21)","(21,7,16)","(20,3,21)","(17,3,24)","(18,5,21)","(20,5,19)"
150,"(40,3,1)","(34,2,8)","(32,2,10)","(16,7,21)","(13,6,25)","(22,6,16)","(23,5,16)","(17,5,22)","(17,7,20)","(21,5,18)","(18,8,18)"
200,"(40,3,1)","(35,3,6)","(33,3,8)","(16,5,23)","(11,5,28)","(18,5,21)","(25,7,12)","(20,5,19)","(18,6,20)","(21,5,18)","(23,4,17)"


In [8]:
dict_to_table['CDB_CLD_mean']

,Standard_Bag_mean,Grouped_Bag_mean,Incre_Bag_mean,CDB_Host_mean,CDB_kDN_mean,CDB_LSC_mean,CDB_N1_mean,CDB_N2_mean,CDB_DCP_mean,CDB_TD_U_mean,CDB_F1_mean
20,"(41,3,0)","(39,3,2)","(37,2,5)","(21,4,19)","(26,4,14)","(20,5,19)","(23,4,17)","(20,7,17)","(21,5,18)","(25,3,16)","(22,3,19)"
30,"(40,3,1)","(39,2,3)","(39,2,3)","(24,5,15)","(29,7,8)","(25,6,13)","(30,3,11)","(25,5,14)","(20,6,18)","(26,4,14)","(25,5,14)"
50,"(41,3,0)","(38,2,4)","(37,2,5)","(22,5,17)","(24,3,17)","(23,6,15)","(30,7,7)","(24,4,16)","(23,4,17)","(25,6,13)","(22,8,14)"
100,"(41,3,0)","(37,2,5)","(34,2,8)","(20,8,16)","(29,4,11)","(21,5,18)","(25,6,13)","(21,9,14)","(21,4,19)","(23,5,16)","(25,5,14)"
150,"(41,3,0)","(35,2,7)","(32,3,9)","(23,8,13)","(25,6,13)","(22,7,15)","(29,5,10)","(23,4,17)","(20,6,18)","(26,4,14)","(25,6,13)"
200,"(41,3,0)","(35,3,6)","(34,2,8)","(22,8,14)","(28,5,11)","(25,6,13)","(30,7,7)","(23,5,16)","(24,5,15)","(23,6,15)","(27,5,12)"


In [9]:
dict_to_table['CDB_LSC_mean']

,Standard_Bag_mean,Grouped_Bag_mean,Incre_Bag_mean,CDB_Host_mean,CDB_kDN_mean,CDB_CLD_mean,CDB_N1_mean,CDB_N2_mean,CDB_DCP_mean,CDB_TD_U_mean,CDB_F1_mean
20,"(41,3,0)","(40,2,2)","(38,2,4)","(15,4,25)","(23,4,17)","(19,5,20)","(23,4,17)","(19,3,22)","(21,4,19)","(20,3,21)","(19,8,17)"
30,"(40,3,1)","(36,3,5)","(38,2,4)","(18,5,21)","(26,5,13)","(13,6,25)","(26,4,14)","(19,7,18)","(20,4,20)","(21,5,18)","(23,6,15)"
50,"(40,3,1)","(36,2,6)","(34,2,8)","(16,4,24)","(22,4,18)","(15,6,23)","(23,4,17)","(22,4,18)","(19,4,21)","(19,5,20)","(21,5,18)"
100,"(39,3,2)","(35,2,7)","(34,4,6)","(18,3,23)","(21,3,20)","(18,5,21)","(22,4,18)","(21,5,18)","(17,3,24)","(22,3,19)","(21,8,15)"
150,"(38,3,3)","(33,2,9)","(31,5,8)","(19,4,21)","(16,6,22)","(15,7,22)","(23,5,16)","(19,5,20)","(17,6,21)","(19,6,19)","(17,6,21)"
200,"(40,3,1)","(33,2,9)","(36,2,6)","(17,5,22)","(21,5,18)","(13,6,25)","(27,4,13)","(23,7,14)","(22,4,18)","(19,6,19)","(23,4,17)"


In [10]:
dict_to_table['CDB_N1_mean']

,Standard_Bag_mean,Grouped_Bag_mean,Incre_Bag_mean,CDB_Host_mean,CDB_kDN_mean,CDB_CLD_mean,CDB_LSC_mean,CDB_N2_mean,CDB_DCP_mean,CDB_TD_U_mean,CDB_F1_mean
20,"(41,3,0)","(39,3,2)","(36,2,6)","(13,3,28)","(21,4,19)","(17,4,23)","(17,4,23)","(17,3,24)","(21,3,20)","(17,6,21)","(22,3,19)"
30,"(39,3,2)","(38,2,4)","(36,2,6)","(16,3,25)","(19,4,21)","(11,3,30)","(14,4,26)","(16,4,24)","(13,5,26)","(14,4,26)","(19,6,19)"
50,"(41,3,0)","(37,2,5)","(33,2,9)","(13,5,26)","(16,6,22)","(7,7,30)","(17,4,23)","(13,4,27)","(15,3,26)","(13,6,25)","(16,4,24)"
100,"(39,3,2)","(35,2,7)","(32,3,9)","(14,5,25)","(16,7,21)","(13,6,25)","(18,4,22)","(16,7,21)","(12,5,27)","(15,6,23)","(20,4,20)"
150,"(39,3,2)","(35,2,7)","(29,2,13)","(12,4,28)","(16,5,23)","(10,5,29)","(16,5,23)","(14,6,24)","(14,3,27)","(13,4,27)","(19,3,22)"
200,"(41,3,0)","(34,3,7)","(32,2,10)","(10,6,28)","(12,7,25)","(7,7,30)","(13,4,27)","(15,6,23)","(15,5,24)","(14,5,25)","(21,5,18)"


In [11]:
dict_to_table['CDB_N2_mean']

,Standard_Bag_mean,Grouped_Bag_mean,Incre_Bag_mean,CDB_Host_mean,CDB_kDN_mean,CDB_CLD_mean,CDB_LSC_mean,CDB_N1_mean,CDB_DCP_mean,CDB_TD_U_mean,CDB_F1_mean
20,"(41,3,0)","(39,2,3)","(39,2,3)","(24,3,17)","(22,6,16)","(17,7,20)","(22,3,19)","(24,3,17)","(23,3,18)","(24,3,17)","(20,4,20)"
30,"(37,3,4)","(38,2,4)","(37,2,5)","(17,3,24)","(20,5,19)","(14,5,25)","(18,7,19)","(24,4,16)","(18,3,23)","(21,3,20)","(24,4,16)"
50,"(41,3,0)","(37,2,5)","(34,2,8)","(15,5,24)","(19,4,21)","(16,4,24)","(18,4,22)","(27,4,13)","(18,4,22)","(21,4,19)","(24,5,15)"
100,"(38,3,3)","(36,2,6)","(33,2,9)","(16,6,22)","(21,3,20)","(14,9,21)","(18,5,21)","(21,7,16)","(14,5,25)","(21,5,18)","(21,5,18)"
150,"(40,3,1)","(37,2,5)","(33,2,9)","(19,7,18)","(22,5,17)","(17,4,23)","(20,5,19)","(24,6,14)","(16,7,21)","(21,6,17)","(22,4,18)"
200,"(39,4,1)","(37,2,5)","(37,2,5)","(15,4,25)","(19,5,20)","(16,5,23)","(14,7,23)","(23,6,15)","(18,3,23)","(22,5,17)","(22,5,17)"


In [12]:
dict_to_table['CDB_DCP_mean']

,Standard_Bag_mean,Grouped_Bag_mean,Incre_Bag_mean,CDB_Host_mean,CDB_kDN_mean,CDB_CLD_mean,CDB_LSC_mean,CDB_N1_mean,CDB_N2_mean,CDB_TD_U_mean,CDB_F1_mean
20,"(41,3,0)","(37,2,5)","(34,2,8)","(20,4,20)","(20,3,21)","(18,5,21)","(19,4,21)","(20,3,21)","(18,3,23)","(19,4,21)","(17,5,22)"
30,"(40,3,1)","(37,2,5)","(38,2,4)","(18,5,21)","(27,6,11)","(18,6,20)","(20,4,20)","(26,5,13)","(23,3,18)","(20,5,19)","(22,6,16)"
50,"(41,3,0)","(39,2,3)","(36,2,6)","(17,6,21)","(22,3,19)","(17,4,23)","(21,4,19)","(26,3,15)","(22,4,18)","(21,4,19)","(22,6,16)"
100,"(39,3,2)","(35,2,7)","(33,2,9)","(23,4,17)","(24,3,17)","(19,4,21)","(24,3,17)","(27,5,12)","(25,5,14)","(25,5,14)","(22,6,16)"
150,"(40,3,1)","(35,2,7)","(32,3,9)","(24,4,16)","(20,7,17)","(18,6,20)","(21,6,17)","(27,3,14)","(21,7,16)","(23,5,16)","(17,9,18)"
200,"(39,4,1)","(32,2,10)","(34,2,8)","(16,4,24)","(20,6,18)","(15,5,24)","(18,4,22)","(24,5,15)","(23,3,18)","(16,3,25)","(19,5,20)"


In [13]:
dict_to_table['CDB_TD_U_mean']

,Standard_Bag_mean,Grouped_Bag_mean,Incre_Bag_mean,CDB_Host_mean,CDB_kDN_mean,CDB_CLD_mean,CDB_LSC_mean,CDB_N1_mean,CDB_N2_mean,CDB_DCP_mean,CDB_F1_mean
20,"(41,3,0)","(40,2,2)","(41,2,1)","(16,4,24)","(20,6,18)","(16,3,25)","(21,3,20)","(21,6,17)","(17,3,24)","(21,4,19)","(22,6,16)"
30,"(40,3,1)","(39,2,3)","(35,2,7)","(16,6,22)","(23,4,17)","(14,4,26)","(18,5,21)","(26,4,14)","(20,3,21)","(19,5,20)","(20,6,18)"
50,"(41,3,0)","(38,2,4)","(37,2,5)","(18,5,21)","(20,3,21)","(13,6,25)","(20,5,19)","(25,6,13)","(19,4,21)","(19,4,21)","(25,6,13)"
100,"(40,4,0)","(36,2,6)","(32,4,8)","(21,3,20)","(21,5,18)","(16,5,23)","(19,3,22)","(23,6,15)","(18,5,21)","(14,5,25)","(25,3,16)"
150,"(39,4,1)","(36,2,6)","(35,3,6)","(17,6,21)","(18,5,21)","(14,4,26)","(19,6,19)","(27,4,13)","(17,6,21)","(16,5,23)","(18,5,21)"
200,"(40,3,1)","(36,2,6)","(34,2,8)","(17,6,21)","(18,5,21)","(15,6,23)","(19,6,19)","(25,5,14)","(17,5,22)","(25,3,16)","(23,4,17)"


In [14]:
dict_to_table['CDB_F1_mean']

,Standard_Bag_mean,Grouped_Bag_mean,Incre_Bag_mean,CDB_Host_mean,CDB_kDN_mean,CDB_CLD_mean,CDB_LSC_mean,CDB_N1_mean,CDB_N2_mean,CDB_DCP_mean,CDB_TD_U_mean
20,"(41,3,0)","(41,2,1)","(35,3,6)","(20,3,21)","(22,5,17)","(19,3,22)","(17,8,19)","(19,3,22)","(20,4,20)","(22,5,17)","(16,6,22)"
30,"(40,3,1)","(39,2,3)","(38,2,4)","(17,4,23)","(21,4,19)","(14,5,25)","(15,6,23)","(19,6,19)","(16,4,24)","(16,6,22)","(18,6,20)"
50,"(41,3,0)","(37,2,5)","(35,2,7)","(15,5,24)","(20,5,19)","(14,8,22)","(18,5,21)","(24,4,16)","(15,5,24)","(16,6,22)","(13,6,25)"
100,"(39,3,2)","(35,3,6)","(33,2,9)","(17,4,23)","(19,5,20)","(14,5,25)","(15,8,21)","(20,4,20)","(18,5,21)","(16,6,22)","(16,3,25)"
150,"(38,4,2)","(37,2,5)","(33,2,9)","(17,7,20)","(18,8,18)","(13,6,25)","(21,6,17)","(22,3,19)","(18,4,22)","(18,9,17)","(21,5,18)"
200,"(40,3,1)","(32,4,8)","(34,4,6)","(15,7,22)","(17,4,23)","(12,5,27)","(17,4,23)","(18,5,21)","(17,5,22)","(20,5,19)","(17,4,23)"
